In [6]:
!pip install -U accelerate
!pip install -U transformers
!pip install torch
!pip install evaluate
!pip install rouge
!pip install rouge_score
!pip install bert_score
!pip install gradio
!pip install typing_extensions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is

In [5]:
from typing_extensions import Doc

In [7]:

import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import csv
from sklearn.model_selection import train_test_split

import gradio as gr
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np

In [10]:

df_train=pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

In [11]:
df_train.head(10)

,question,answer
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor..."
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...
5,What is the mean absolute error (MAE)?,The mean absolute error (MAE) measures the ave...
6,What is the Epsilon-Greedy Algorithm?,The epsilon-greedy algorithm is a method for b...
7,What is precision in the context of performanc...,Precision measures the ratio of correctly pred...
8,"What is an encoder-only transformer model, and...",Encoder-only transformer models like BERT are ...
9,How does regularization affect the loss function?,Regularization adds a penalty term to the loss...


In [12]:
df_test.head(10)

,question,answer
0,What is feature selection?,The process of selecting a subset of relevant ...
1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...
2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch
3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...
4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...
5,What is a Decision Tree?,A Decision Tree is a predictive learning algor...
6,What is variance in machine learning?,Variance refers to the error introduced by the...
7,What are common methods to diagnose bias and v...,Common methods include analyzing learning curv...
8,What is the difference between L1 and L2 regul...,L1 regularization adds the absolute values of ...
9,What is feature engineering for unstructured d...,Feature engineering for unstructured data invo...


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "gpt2"  #"gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained(checkpoint, pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [15]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

# default to left padding
tokenizer.padding_side = "left"
tokenizer.pad_token=tokenizer.eos_token

In [17]:
from torch.nn import functional as F

max_seq_length = 512

tokenized_data = []
for _, row in df_train.iterrows():
    question = row['question']
    answer = row['answer']
    question = " ".join(question.split()[:300])
    out = "<Question>\n" + str(question) + " \nAnswer\n " + str(answer) +" \n<ANSWER_ENDED>\n "
    out_ids = tokenizer(out,truncation = True, max_length = max_seq_length)


    tokenized_data.append(out_ids)



In [18]:
class CustomTextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

# Create a custom dataset
custom_dataset = CustomTextDataset(tokenized_data)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [19]:
# Training arguments
training_args = TrainingArguments(
    output_dir='/content/gpt_model',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
)

In [20]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset,
)

In [21]:
# Train the model
trainer.train()

Step,Training Loss
500,2.126000
1000,1.596100


TrainOutput(global_step=1491, training_loss=1.7171063899674086, metrics={'train_runtime': 223.3163, 'train_samples_per_second': 26.666, 'train_steps_per_second': 6.677, 'total_flos': 228907130112000.0, 'train_loss': 1.7171063899674086, 'epoch': 3.0})

In [22]:
model_output_path = "/content/drive/MyDrive/QA_MODEL_D2/model"

In [23]:
 # Save the model
trainer.save_model(model_output_path)
# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

('/content/drive/MyDrive/QA_MODEL_D2/model/tokenizer_config.json',
 '/content/drive/MyDrive/QA_MODEL_D2/model/special_tokens_map.json',
 '/content/drive/MyDrive/QA_MODEL_D2/model/vocab.json',
 '/content/drive/MyDrive/QA_MODEL_D2/model/merges.txt',
 '/content/drive/MyDrive/QA_MODEL_D2/model/added_tokens.json')

In [24]:
def generate_answer(question):
    input_ids = tokenizer.encode(question, return_tensors="pt").to("cuda")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids).to("cuda")
    pad_token_id = tokenizer.eos_token_id

    #output = model[0].generate(
    output = model.generate(
        input_ids,
        max_new_tokens=100,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    start_index = decoded_output.find("Answer")
    end_index = decoded_output.find("<ANSWER_ENDED>")

    if end_index != -1:
        # Extract the text between "Answer" and "<ANSWER_ENDED>"
        answer_text = decoded_output[start_index + len("Answer"):end_index].strip()
        return answer_text
    else:
        # If "<ANSWER_ENDED>" is not found, return the text following "Answer"
        answer_text = decoded_output[start_index + len("Answer"):].strip()
        return answer_text


In [25]:
model_output_path='/content/drive/MyDrive/QA_MODEL_D2/model'

In [26]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_output_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

In [28]:
#model[0].to("cuda")
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [29]:
question="What is deep learning?"
response = generate_answer(question)
print("Generated Answer:", response)

Generated Answer: Deep learning is a type of artificial intelligence that uses deep learning algorithms to learn and interpret data.


In [30]:
from tqdm import tqdm
tqdm.pandas(desc="Generating answers")
df_test['generated_answers'] = df_test['question'].progress_apply(lambda question: generate_answer(question))

Generating answers: 100%|██████████| 249/249 [03:41<00:00,  1.12it/s]


In [31]:
df_test.head(5)

,question,answer,generated_answers
0,What is feature selection?,The process of selecting a subset of relevant ...,Feature selection is the process of selecting ...
1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...,MDP involves predicting the next action in a s...
2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch,Batch normalization involves dividing the data...
3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...,Word2Vec creates word embeddings by combining ...
4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...,Hidden layers in MLP allow the model to learn ...


In [32]:
import re

def extract_answer(text):
    pattern = r'Answer(.*?)<ANSWER_ENDED>'
    result = re.search(pattern, text, re.DOTALL)

    if result:
        return result.group(1).strip()
    else:
        return text

In [33]:
df_test['generated_answers'] = df_test['generated_answers'].apply(extract_answer)

In [34]:
df_test.head(5)

,question,answer,generated_answers
0,What is feature selection?,The process of selecting a subset of relevant ...,Feature selection is the process of selecting ...
1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...,MDP involves predicting the next action in a s...
2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch,Batch normalization involves dividing the data...
3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...,Word2Vec creates word embeddings by combining ...
4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...,Hidden layers in MLP allow the model to learn ...


In [35]:
df_test.to_csv('model_Qtest_data_output.csv', index=False)

In [36]:
df_test=pd.read_csv('/content/model_Qtest_data_output.csv')

In [37]:
df_test.columns

Index(['question', 'answer', 'generated_answers'], dtype='object')

In [40]:
from evaluate import load
from collections import defaultdict
from tqdm import tqdm
from bert_score import score

def metric(data):
    Question,Answer,Generated_Answer  = ["question","answer","generated_answers"]
    rouge = load("rouge")
    meteor = load("meteor")
    score_bert = defaultdict(int)
    bertscore = load("bertscore")

    for _,x in tqdm(data.iterrows(),total=len(data)):

        rouge_ans = rouge.compute(predictions=[x[Generated_Answer]],references=[x[Answer]])

        score_bert["rouge_1_ans"] += rouge_ans['rouge1']
        score_bert["rouge_2_ans"] += rouge_ans['rouge2']
        score_bert["rouge_L_ans"] += rouge_ans['rougeL']

        meteor_ans = meteor.compute(predictions=[x[Generated_Answer]],references=[x[Answer]])

        score_bert["meteor_ans"] += meteor_ans["meteor"]

        # BERT score
        results = bertscore.compute(predictions=[x[Generated_Answer]], references=[x[Answer]], lang="en")

        score_bert["bert_score_f1"] += results["f1"][0]

    print("The evalution scores are as follows:")

    for name,val in score_bert.items():
        print(f"{name}: {val/len(data)}")




In [41]:
metric(df_test)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
  0%|          | 0/249 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 249/249 [01:16<00:00,  3.25it/s]

The evalution scores are as follows:
rouge_1_ans: 0.30352747986275347
rouge_2_ans: 0.12017356938462426
rouge_L_ans: 0.2623553379781145
meteor_ans: 0.26038046040240764
bert_score_f1: 0.8836348801252832
